In [90]:
import sys
sys.path.insert(0, '..')

import json
import tweepy

from paradeller.keys import (
    consumer_key, consumer_secret, access_token, accss_token_secret
)

from paradeller.stopwords import stopwords

from tqdm import tqdm, trange

In [2]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, accss_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [3]:
def is_good(t):
    # low word count
    tweet_len = len(t.split())
    if (tweet_len < 3) or (tweet_len >= 10):
        return False
    
    # no blacklistd works / symbols
    for x in ["@", "\n"]:
        if x in t:
            return False
        
    return True

In [4]:
def display_status(status):
    print(status.text)
    print(f"-- @{status.author.screen_name}\n")
    print(status.id)
    print("~"*50)

In [11]:
def get_tweets():
    query = "-filter:retweets -filter:links -filter:media"
    tweets = api.search(
        q=query, lang='en', count=100, include_entities=False
    )
    return [t for t in tweets if is_good(t.text)]

In [12]:
res = get_tweets()

for status in res:
    display_status(status)

new Boris is lit
-- @StudentOfEth

1144356141872271360
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
I been sleep! I was tired asf
-- @x_PinkNPearls

1144356141868232713
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
sometimes I wish I had hoes
-- @fauxlavie

1144356141843066880
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Lets go torrejon
-- @OmFetty

1144356141817913344
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ALL OF THEM
-- @stankrhodes

1144356141713018880
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
these ain’t hickeys baby you know my chains fake
-- @SophieA24683

1144356141671100431
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Sometimes you just have to take the L
-- @CFeli19

1144356141549465614
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Creaturos - Last summers' #1 Hit Single
-- @WZBC_Live

1144356141503266816
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Rant over! I'm just angry 😡
-- @IllestDJCarpel

1144356141503143936
~~~~~

### Data Collection

In [13]:
from paradeller.helper import load_archive, update_archive

In [14]:
def save_status(status):
    return dict(
        id=status.id,
        text=status.text,
        author=status.author.screen_name,
        time=status.created_at.strftime("%Y-%m-%d %H:%M:%S")
    )

In [91]:
archive = load_archive()
len(archive)

51

In [92]:
n = 1000
for i in trange(n):    
    # get tweets
    statuses = get_tweets()
    
    # get into save format
    tweets = [save_status(s) for s in statuses]

    # add to archive list
    archive.extend(tweets)

    
# save to file
update_archive(archive)

100%|██████████| 1000/1000 [1:05:25<00:00,  1.74it/s]  


10206

In [93]:
archive = load_archive()
len(archive)

10206

## Selected

In [8]:
def save_status(status):
    return dict(
        id=status.id,
        text=status.text,
        author=status.author.screen_name,
        time=status.created_at
    )

In [9]:
def save_status_from_id(id_):
    status = api.get_status(id_)
    return save_status(status)

In [10]:
ids = [
    1144127521929224197,
    1144135638721036288,
    1144135638733557760,
    1144137985971183616,
    1144144954219999232,
    1144146074321317888,
    1144313057377546240,
    1144313057444847621
]

In [11]:
saved = []
for id_ in ids:
    try:
        s = save_status_from_id(id_)
        saved.append(s)
    except tweepy.TweepError as e:
        print(e)
        print(">> ", id_)

In [12]:
saved

[{'id': 1144127521929224197,
  'text': 'Sitting in lingerie drinking wine ima wife for sure 😭',
  'author': '__mxvii',
  'time': datetime.datetime(2019, 6, 27, 6, 17, 12)},
 {'id': 1144135638721036288,
  'text': 'time will be frozen for us',
  'author': 'mariuuhhhh',
  'time': datetime.datetime(2019, 6, 27, 6, 49, 27)},
 {'id': 1144135638733557760,
  'text': 'Is it me or is it a break up season?',
  'author': 'Linnneyyy',
  'time': datetime.datetime(2019, 6, 27, 6, 49, 27)},
 {'id': 1144137985971183616,
  'text': 'ive been gettin horny at the most inconvenient times lately wtf 🥴😫',
  'author': 'SoCvsh',
  'time': datetime.datetime(2019, 6, 27, 6, 58, 47)},
 {'id': 1144144954219999232,
  'text': 'Fuck work experience 🖕🏾🖕🏾',
  'author': 'juggz_19',
  'time': datetime.datetime(2019, 6, 27, 7, 26, 28)},
 {'id': 1144146074321317888,
  'text': 'All you can do is watch.',
  'author': 'ayeroven',
  'time': datetime.datetime(2019, 6, 27, 7, 30, 55)},
 {'id': 1144313057377546240,
  'text': 'Pier

## Searching for matches

In [13]:
import emoji
import string
from tqdm import tqdm, trange

In [14]:
def tokenize(tweet):
    tweet = ' '.join(emoji.get_emoji_regexp().split(tweet))
    words = tweet.split()
    return [
        w.lower().strip().translate(str.maketrans('', '', string.punctuation))
        for w in words
    ]

In [15]:
tokenize(saved[0]['text'])

['sitting',
 'in',
 'lingerie',
 'drinking',
 'wine',
 'ima',
 'wife',
 'for',
 'sure',
 '😭']

### Pick some tweets

In [16]:
s1 = saved[0]
s1

{'id': 1144127521929224197,
 'text': 'Sitting in lingerie drinking wine ima wife for sure 😭',
 'author': '__mxvii',
 'time': datetime.datetime(2019, 6, 27, 6, 17, 12)}

In [17]:
s2 = saved[7]
s2

{'id': 1144313057444847621,
 'text': 'did I mention i’m gay?',
 'author': 'hoeslynn',
 'time': datetime.datetime(2019, 6, 27, 18, 34, 27)}

In [18]:
word_set = set(tokenize(s1['text']) + tokenize(s2['text']))

In [19]:
word_set

{'did',
 'drinking',
 'for',
 'gay',
 'i',
 'ima',
 'in',
 'i’m',
 'lingerie',
 'mention',
 'sitting',
 'sure',
 'wife',
 'wine',
 '😭'}

In [20]:
search_words = word_set - set(stopwords)
search_words

{'did',
 'drinking',
 'gay',
 'i',
 'ima',
 'i’m',
 'lingerie',
 'mention',
 'sitting',
 'sure',
 'wife',
 'wine',
 '😭'}

In [31]:
search_words

{'did',
 'drinking',
 'gay',
 'i',
 'ima',
 'i’m',
 'lingerie',
 'mention',
 'sitting',
 'sure',
 'wife',
 'wine',
 '😭'}

In [32]:
search_words

{'did',
 'drinking',
 'gay',
 'i',
 'ima',
 'i’m',
 'lingerie',
 'mention',
 'sitting',
 'sure',
 'wife',
 'wine',
 '😭'}

In [21]:
query = (
    ' OR '.join(search_words) + 
    " -filter:retweets -filter:links -filter:media"
)
print(query)

def get_search_tweets():
    tweets = api.search(
        q=query, lang='en', count=1000, include_entities=False
    )
    return [t for t in tweets if is_good(t.text)]

sure OR did OR 😭 OR mention OR sitting OR wine OR i OR wife OR lingerie OR ima OR drinking OR gay OR i’m -filter:retweets -filter:links -filter:media


In [25]:
def potential_match(tweet, word_set):
    """
    If all words in tweet are in the word set,
    it is a potential match
    """
    words = tokenize(tweet)
    return set(words) <= word_set

In [34]:
iterations = 1000

matches = []

for i in trange(iterations):
    res = get_search_tweets()
    for status in res:
        if potential_match(status.text, word_set):
            print("yeah!!")
            display_status(status)
            mactches.append(status)

100%|██████████| 1000/1000 [1:09:07<00:00,  1.36it/s]   


In [36]:
matches

[]